### Lab Assignment: Commercial Data Analysis

### University of Virginia
### DS 7200: Distributed Computing
### Last Updated: August 20, 2023

---

### INSTRUCTIONS  
In this assignment, you will work with a dataset containing information about businesses.  
Each record is a business location.  Follow the steps below, writing and running the code in blocks, and displaying the solutions.  

Each question part is worth 1 POINT, for a total of 15 POINTS.

Hint: reaching deeper fields in json hierarchy can be done like this:  

`df.select('address.street_number')`

---

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("comm") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/20 11:35:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/20 11:35:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/20 11:35:01 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/09/20 11:35:01 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/09/20 11:35:01 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
# note that read.json can read a zipped JSON directly

In [6]:
from pyspark.sql.functions import col

**1. (1 PT) Read in the dataset and show the number of records**

In [3]:
filename = "/standard/ds7200-apt4c/large_datasets/part-00000-a159c41a-bc58-4476-9b78-c437667f9c2b-c000.json.gz"
df = spark.read.json(filename)

24/09/20 11:35:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
df.count()

154679

**2. (1 PT) Show the first 5 records**

In [40]:
df.show(5)

+--------------------+--------------------+--------------------+----------------+----+--------------------+--------------------+--------------------+
|             address|       business_tags|               hours|              id|menu|             reviews|                urls|             webpage|
+--------------------+--------------------+--------------------+----------------+----+--------------------+--------------------+--------------------+
|{Woodburn, {45.15...|                null|                null|000023995a540868|null|                  []|{woodburn.k12.or....|{Educational Tech...|
|{Hialeah, {25.884...|{[], [{has_atm, Y...|{null, 1900, null...|0000821a1394916e|null|                null|{null, [yelp.com]...|                null|
|{Rochester, {43.1...|{[], [{accepts_cr...|{null, 1700, null...|000136e65d50c3b7|null|[{New (to me) qui...|{usps.com, [yelp....|{Welcome | USPS G...|
|{West Palm Beach,...|                null|                null|00014329a70b9869|null|              

**3. (1 PT) Show the first 5 street addresses which are not null**  

In [8]:
df.select(col('address')).filter(col('address').isNotNull()).show(5)

+--------------------+
|             address|
+--------------------+
|{Woodburn, {45.15...|
|{Hialeah, {25.884...|
|{Rochester, {43.1...|
|{West Palm Beach,...|
|{Eufaula, {35.283...|
+--------------------+
only showing top 5 rows



**4. (1 PT) Location**  

Count the number of records where the city is Phoenix

In [24]:
df.filter(col('address.city') == "Phoenix").count()

762

**5. (1 PT) Hours**  

Count the number of records where closing time on Thursday is 8pm

In [54]:
df.filter(col('hours.thursday_close') == 2000).count()

3313

**6. (1 PT) Location and Hours**  

Count the number of records where city is Phoenix and closing time on Thursday is 8pm

In [36]:
df.filter((col('address.city') == "Phoenix") & (col('hours.thursday_close') == 2000)).count()

12

**7. (1 PT) Price Range**  

Price range is quoted in number of dollar signs.  Count the number of records with price range greater than or equal to two.

Note: most of the price ranges are null, which is annoying

In [52]:
df.filter(col('menu.price_range') >= 2).count()

1135

**8. (1 PT) COMPANY HEADQUARTERS**  

For the `address.is_headquarters` field:  
how many locations are HQ / are NOT HQ / are null?

In [60]:
df.groupBy(col('address.is_headquarters')).count().show()

+---------------+-----+
|is_headquarters|count|
+---------------+-----+
|           null|87625|
|           true|  318|
|          false|66736|
+---------------+-----+



**9. (1 PT) Webpage URLs**  

Register the dataframe as a temp table.  
Next, use Spark SQL to select only the webpage title column, filtering on rows where the webpage url (accessed under `webpage.url`) is *Target.com*. 

Show only one resulting row and don't truncate the output.

In [67]:
df.createOrReplaceTempView("commerical")

spark.sql("SELECT webpage.title FROM commerical where webpage.url == 'Target.com'").show(1, truncate=False)

+-------------------------------+
|title                          |
+-------------------------------+
|Target : Expect More. Pay Less.|
+-------------------------------+
only showing top 1 row



**10. (1 PT) Analysis on Ratings**  

The reviews contains information such as the number of stars for each review (the *rating*).  
The ratings are stored in an array (`reviews.stars`) for each business location (you should check for yourself). Return the top five most common rating arrays.  For example, an array might look like: 
[5, 5]



*If we include `null` values*:

In [27]:
df.groupBy(col('reviews.stars')).count().orderBy('count', ascending=False).show(5)

+------+-----+
| stars|count|
+------+-----+
|  null|74679|
|    []|42419|
|   [5]| 4258|
|[null]| 3067|
|[5, 5]| 1610|
+------+-----+
only showing top 5 rows



-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

*If we **ignore** `null` values*:

In [30]:
from pyspark.sql.functions import exists

df.filter(exists(col('reviews.stars'), lambda x: x.isNotNull())).groupBy(col('reviews.stars')).count().sort(col('count').desc()).show(5)

+---------+-----+
|    stars|count|
+---------+-----+
|      [5]| 4258|
|   [5, 5]| 1610|
|      [1]| 1559|
|[5, 5, 5]|  836|
|      [4]|  776|
+---------+-----+
only showing top 5 rows



**11. More work with Ratings**  

For this question, you will filter out null ratings and then compute the average rating for each business location (using the field: `id`).


a) (1 PT) Create a new dataframe retaining two fields: `id`, `reviews.stars`


In [7]:
stars_df = df.select(col('id'), col('reviews.stars'))

b) (1 PT) Create a row for each rating  
hint: use the `withColumn()` and `explode()` functions  
you will need to import the `explode()` function by issuing:

`from pyspark.sql.functions import explode`


In [8]:
from pyspark.sql.functions import explode

stars_df_exploded = stars_df.withColumn('stars', explode(col('stars')))
stars_df_exploded.show()

+----------------+-----+
|              id|stars|
+----------------+-----+
|000136e65d50c3b7|    4|
|000136e65d50c3b7|    4|
|0003b7589a4e12a0|    5|
|00045f958e4bb02a| null|
|00045f958e4bb02a| null|
|00045f958e4bb02a| null|
|00045f958e4bb02a| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4|    1|
|00059519f0dba1b4|    5|
|00059519f0dba1b4|    2|
|00059519f0dba1b4|    4|
|00059519f0dba1b4|    5|
+----------------+-----+
only showing top 20 rows



c) (1 PT) Return a count of the number of ratings in this dataframe

In [10]:
stars_df_exploded.count()

600082

d) (1 PT) Drop rows where the rating is null, and return a count of the number of non-null ratings

In [16]:
stars_df_exploded_clean = stars_df_exploded.filter(col('stars').isNotNull())
stars_df_exploded_clean.count()

538241

e) (1 PT) Compute the average rating, grouped by `id`. After the average is computed, sort by `id` in ascending order and show the top 10 records.  
 
hint:   
this can all be done in one line using the `agg()` function  
this `id` should be at the top: 000136e65d50c3b7

In [21]:
from pyspark.sql.functions import avg

# sorting by ascending id is the default behavior, so we don't need to specify that part
# not sure why we'd need the agg() function since we can just the avg directly to average one column (stars col)
stars_df_exploded_clean.groupBy(col('id')).avg().show(10)

+----------------+------------------+
|              id|        avg(stars)|
+----------------+------------------+
|000136e65d50c3b7|               4.0|
|0003b7589a4e12a0|               5.0|
|00059519f0dba1b4|3.3333333333333335|
|000a1df4c8e0ecd2|               4.6|
|000c7b7a30623083|               5.0|
|000c9ffc8b89af03|               3.0|
|000de20baa847ecc|1.6666666666666667|
|001064359d9f162f|               5.0|
|0010c9f495d87dd7|               3.0|
|0017774db5e6400a| 4.333333333333333|
+----------------+------------------+
only showing top 10 rows

